#### Use this for sentiment analysis with Logistic Regression and Naive Bayes
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/1_IMPORTANT_Capstone-Project-master/Capstone-Project-master/Capstone_Project_Sentiment_Analysis.ipynb
https://github.com/santhoshidharmireddy/Capstone-Project/blob/master/Capstone_Project_Sentiment_Analysis.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/1_IMPORTANT_Capstone-Project-master/Capstone-Project-master/Capstone_Project_KNN_RecommenderSystem.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/2_Amazon-Product-Recommender-System-master/Naive%20Bayes/Amazon%20reviews_Naive%20Bayes.ipynb - USING THIS FOR NAIVE BAYES
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/2_Amazon-Product-Recommender-System-master/Recommender%20System/Recommender%20System.ipynb

Removing Contractions - https://medium.com/analytics-vidhya/sentiment-analysis-on-amazon-reviews-using-tf-idf-approach-c5ab4c36e7a1


http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Recommendation%2BSystem%2BNotebook_WIP.ipynb
http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Recommendation%2BSystem%2BNotebook.ipynb

http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Akash_collaborative_user_base..ipynb

Random Forest - http://localhost:8888/notebooks/_PythonNotebooks/_Course4_Machine_Learning_2/Mod4_Tree_Models/TelecomChurn/Telecom%2BChurn%2B-%2BLogistic%2B%2B%2BDT%2B%2B%2BRF.ipynb



# Sentiment-based product recommendation system for Ebuss
### Author - Kumaraguru Muthuraj

## Install the required libraries

In [2]:
!pip install textblob
!pip install wordcloud

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

In [4]:
import re

In [5]:
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn import neighbors
import matplotlib.pyplot as plt

In [6]:
from string import punctuation
from sklearn import svm
from textblob import Word
from wordcloud import WordCloud
import textblob
from textblob import TextBlob

In [7]:
from imblearn.over_sampling import SMOTE

## Task 1: Data Cleaning and Pre-Processing

### Lets do some EDA. Load the data. Check for the missing values. Treat them. Impute if necessary or drop the fields. Get a sense of data imbalance too.

In [8]:
reviews_df = pd.read_csv("sample30.csv")

In [9]:
print(type(reviews_df))

<class 'pandas.core.frame.DataFrame'>


In [10]:
print(reviews_df.columns)
print(reviews_df.shape)

Index(['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews_date',
       'reviews_didPurchase', 'reviews_doRecommend', 'reviews_rating',
       'reviews_text', 'reviews_title', 'reviews_userCity',
       'reviews_userProvince', 'reviews_username', 'user_sentiment'],
      dtype='object')
(30000, 15)


### We need only the 'name', 'reviews_rating', 'reviews_text', 'reviews_username' and 'user_sentiment'. Rest of the columns can be dropped.

In [11]:
reviews_df = reviews_df.drop(['id', 'brand', 'categories', 'manufacturer', 
                              'reviews_date', 'reviews_didPurchase', 'reviews_doRecommend', 
                              'reviews_title', 'reviews_userCity', 'reviews_userProvince'], axis=1)

In [12]:
reviews_df

,name,reviews_rating,reviews_text,reviews_username,user_sentiment
0,Pink Friday: Roman Reloaded Re-Up (w/dvd),5,i love this album. it's very good. more to the...,joshua,Positive
1,Lundberg Organic Cinnamon Toast Rice Cakes,5,Good flavor. This review was collected as part...,dorothy w,Positive
2,Lundberg Organic Cinnamon Toast Rice Cakes,5,Good flavor.,dorothy w,Positive
3,K-Y Love Sensuality Pleasure Gel,1,I read through the reviews on here before look...,rebecca,Negative
4,K-Y Love Sensuality Pleasure Gel,1,My husband bought this gel for us. The gel cau...,walker557,Negative
...,...,...,...,...,...
29995,L'or233al Paris Elvive Extraordinary Clay Reba...,5,I got this conditioner with Influenster to try...,laurasnchz,Positive
29996,L'or233al Paris Elvive Extraordinary Clay Reba...,5,"I love it , I received this for review purpose...",scarlepadilla,Positive
29997,L'or233al Paris Elvive Extraordinary Clay Reba...,5,First of all I love the smell of this product....,liviasuexo,Positive
29998,L'or233al Paris Elvive Extraordinary Clay Reba...,5,I received this through Influenster and will n...,ktreed95,Positive


#### There are ~25K users in in the dataset.

In [13]:
reviews_df['reviews_username'].value_counts()

mike                 41
byamazon customer    41
chris                32
lisa                 16
rick                 15
                     ..
cowboy_yank           1
cloroxisgreat         1
hawthorne             1
rjejr                 1
loveshopping03        1
Name: reviews_username, Length: 24914, dtype: int64

#### There are 271 unique products in the dataset.

In [14]:
reviews_df['name'].value_counts()

Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                         8545
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd                   3325
Clorox Disinfecting Bathroom Cleaner                                              2039
L'or233al Paris Elvive Extraordinary Clay Rebalancing Conditioner - 12.6 Fl Oz    1186
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)               1143
                                                                                  ... 
Progresso Traditional Chicken Rice With Vegetables Soup                              1
Noosa Honey Yogurt                                                                   1
Black Sister's Revenge (dvd)                                                         1
Iman Luxury Moisturizing Lipstick, Black Brandy 006                                  1
Herr's Baked Cheese Curls                                                            1
Name: name, Length: 271, dtype: int64

In [15]:
reviews_df['reviews_username'] = reviews_df['reviews_username'].str.lower()

In [16]:
reviews_df.head()

,name,reviews_rating,reviews_text,reviews_username,user_sentiment
0,Pink Friday: Roman Reloaded Re-Up (w/dvd),5,i love this album. it's very good. more to the...,joshua,Positive
1,Lundberg Organic Cinnamon Toast Rice Cakes,5,Good flavor. This review was collected as part...,dorothy w,Positive
2,Lundberg Organic Cinnamon Toast Rice Cakes,5,Good flavor.,dorothy w,Positive
3,K-Y Love Sensuality Pleasure Gel,1,I read through the reviews on here before look...,rebecca,Negative
4,K-Y Love Sensuality Pleasure Gel,1,My husband bought this gel for us. The gel cau...,walker557,Negative


#### There are no empty review rows.

In [17]:
print(reviews_df['reviews_text'].isnull().sum())

0


#### There are no empty review ratings.

In [18]:
print(reviews_df['reviews_rating'].isnull().sum())

0


### There are no records to be imputed or dropped because all of them seem to have values. We don't have to drop records for products. At the same time, the below list shows that there are around ~50 products that have just 1 reviews. These can be dropped as they qualify to be outliers, but we choose to retain them.

In [19]:
l = reviews_df['name'].value_counts().sort_values()[0:60]
l

Herr's Baked Cheese Curls                                                                               1
Walkers Stem Ginger Shortbread                                                                          1
Fantasy Fields Lil' Sports Fan Step Stool - Teamson                                                     1
Sea Gull Lighting Ceiling Fan - White                                                                   1
Progresso Traditional Meatball & Rice Soup                                                              1
5302050 15/16 FCT/HOSE ADAPTOR                                                                          1
Disney174 Jake And The Neverland Pirates 4 Piece Bedding Set - Toddler                                  1
Germ Guardian174 Elite 3-In-1 Pet Pure True Hepa Air Purifier Ac4300bptca                               1
Citrus Magic Instant Spot & Stain Remover                                                               1
Udi's Pepperoni Pizza                         

In [20]:
reviews_df['reviews_text']

0        i love this album. it's very good. more to the...
1        Good flavor. This review was collected as part...
2                                             Good flavor.
3        I read through the reviews on here before look...
4        My husband bought this gel for us. The gel cau...
                               ...                        
29995    I got this conditioner with Influenster to try...
29996    I love it , I received this for review purpose...
29997    First of all I love the smell of this product....
29998    I received this through Influenster and will n...
29999    I received this product complimentary from inf...
Name: reviews_text, Length: 30000, dtype: object

## Task 2: Text Processing

### Check if the reviews have any hypertext references.

In [21]:
def checkHyperText(x):
    if x.find("http:") == -1:
        return 0
    else:
        return 1
    
http = reviews_df['reviews_text'].apply(checkHyperText)
http.value_counts()

0    29994
1        6
Name: reviews_text, dtype: int64

### There are a few, hyperlinks, but they are descriptive. Remove the hyperlinks in the reviews text.

In [22]:
reviews_df['reviews_tokenized'] = reviews_df['reviews_text'].apply(lambda x: re.sub(r"http\S+", "", x))
reviews_df['reviews_tokenized']

0        i love this album. it's very good. more to the...
1        Good flavor. This review was collected as part...
2                                             Good flavor.
3        I read through the reviews on here before look...
4        My husband bought this gel for us. The gel cau...
                               ...                        
29995    I got this conditioner with Influenster to try...
29996    I love it , I received this for review purpose...
29997    First of all I love the smell of this product....
29998    I received this through Influenster and will n...
29999    I received this product complimentary from inf...
Name: reviews_tokenized, Length: 30000, dtype: object

### I applied Lemmatization and found that it had a negative impact on the review classification and the accuracy came down. I choose to convert contractions programmatically - explicitly.

In [23]:
def expand_contractions(s):
    s = re.sub(r"won't", "will not", s)
    s = re.sub(r"would't", "would not", s)
    s = re.sub(r"could't", "could not", s)
    s = re.sub(r"\'d", " would", s)
    s = re.sub(r"can\'t", "can not", s)
    s = re.sub(r"n\'t", " not", s)
    s= re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s

reviews_df['reviews_tokenized'] =reviews_df['reviews_tokenized'].apply(lambda x:expand_contractions(x))
reviews_df['reviews_tokenized']

0        i love this album. it is very good. more to th...
1        Good flavor. This review was collected as part...
2                                             Good flavor.
3        I read through the reviews on here before look...
4        My husband bought this gel for us. The gel cau...
                               ...                        
29995    I got this conditioner with Influenster to try...
29996    I love it , I received this for review purpose...
29997    First of all I love the smell of this product....
29998    I received this through Influenster and will n...
29999    I received this product complimentary from inf...
Name: reviews_tokenized, Length: 30000, dtype: object

### Tokenize the reviews text and retain only the English alphabet - words.

In [24]:
#Tokenize the review text to words
reg = re.compile('[^a-zA-Z]+')
def tokenizeReview(text):
    text = text.lower()
    text = reg.sub(' ', text).strip()
    return text

reviews_df['reviews_tokenized'] = reviews_df['reviews_tokenized'].apply(tokenizeReview)
reviews_df['reviews_tokenized']

0        i love this album it is very good more to the ...
1        good flavor this review was collected as part ...
2                                              good flavor
3        i read through the reviews on here before look...
4        my husband bought this gel for us the gel caus...
                               ...                        
29995    i got this conditioner with influenster to try...
29996    i love it i received this for review purposes ...
29997    first of all i love the smell of this product ...
29998    i received this through influenster and will n...
29999    i received this product complimentary from inf...
Name: reviews_tokenized, Length: 30000, dtype: object

### Compare the original reviews text and pre-processed reviews text that will be fed to the next stage.

In [25]:
reviews_df[['reviews_text', 'reviews_tokenized']]

,reviews_text,reviews_tokenized
0,i love this album. it's very good. more to the...,i love this album it is very good more to the ...
1,Good flavor. This review was collected as part...,good flavor this review was collected as part ...
2,Good flavor.,good flavor
3,I read through the reviews on here before look...,i read through the reviews on here before look...
4,My husband bought this gel for us. The gel cau...,my husband bought this gel for us the gel caus...
...,...,...
29995,I got this conditioner with Influenster to try...,i got this conditioner with influenster to try...
29996,"I love it , I received this for review purpose...",i love it i received this for review purposes ...
29997,First of all I love the smell of this product....,first of all i love the smell of this product ...
29998,I received this through Influenster and will n...,i received this through influenster and will n...


In [26]:
reviews_df.shape

(30000, 6)

In [27]:
reviews_df = reviews_df.drop(['reviews_text'], axis=1)

In [28]:
reviews_df.shape

(30000, 5)

In [29]:
reviews_df['reviews_tokenized'].head(5)

0    i love this album it is very good more to the ...
1    good flavor this review was collected as part ...
2                                          good flavor
3    i read through the reviews on here before look...
4    my husband bought this gel for us the gel caus...
Name: reviews_tokenized, dtype: object

### user_sentiment column as 89% positive reviews and just 11% negative reviews. This is huge imbalance in data that would affect the learning process.

In [30]:
reviews_df['user_sentiment'].value_counts()

Positive    26632
Negative     3367
Name: user_sentiment, dtype: int64

In [31]:
# Convert 'positive' to 1 and 'negative' to 0
def sentiment2Number(txt):
    if str(txt).lower() == 'positive':
        return 1
    else:
        return 0
    
X = reviews_df['reviews_tokenized']
y = reviews_df['user_sentiment'].apply(sentiment2Number)

In [32]:
print(len(X))
print(len(y))

30000
30000


In [33]:
y

0        1
1        1
2        1
3        0
4        0
        ..
29995    1
29996    1
29997    1
29998    1
29999    1
Name: user_sentiment, Length: 30000, dtype: int64

## Task 3: Feature Extraction

### Use TF-IDF Vectorizer and leverage the built-in stop_words functionality and remove stop words too. Use the max_features as a hyper-parameter.

In [34]:
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words= 'english')

#### As indicated earlier, we have a highly skewed data-set, which is not in balance.

In [35]:
#Indicates class imbalance
y.value_counts()

1    26632
0     3368
Name: user_sentiment, dtype: int64

### Train and test data split.
### Use SMOTE to fix class imbalance before training the model.

In [36]:
#Handle class imbalance
X_features = tfidf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=67)
sm = SMOTE()
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)
X_test_smote, y_test_smote = sm.fit_resample(X_test, y_test)

### Core function of the project. Fit the tf-idf vectorized features into the text classification model. Evaluate it with the ROC - AUC method.

In [37]:
def class_balanced_model_fit(X_train_smote, y_train_smote, X_test_smote, y_test_smote, ml_model, coef_show):
    classification = ml_model.fit(X_train_smote, y_train_smote)
    classification_pred = classification.predict(X_test_smote)
    accuracy = classification.score(X_test_smote, y_test_smote)
    model_performance = classification_report(y_test_smote, classification_pred)
    validation_pred_proba_grad = classification.predict_proba(X_test_smote)
    roc_auc = roc_auc_score(y_test_smote, validation_pred_proba_grad[:,1])
    
    print("***** Accuracy of the classification model: ", accuracy, " *******")
    print('')
    print(model_performance)
    print('')
    print("***** ROC_AUC score: ", roc_auc, " *******")
    print("*************************************************")
    
    if coef_show == 1:
        featureNames = tfidf.get_feature_names()
        coef = classification.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : featureNames, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print("************ Top 10 positive features (variables) ************")
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print("************ Top 10 negative features (variables) ************")        
        print(coeff_df.tail(20).to_string(index=False))
    
    return classification

## Task 4: Model Building, with 3 models - Logistic Regression, Bayes and Random Forest.

### Use Logistic Regression as the classification model and evaluate.
#### Accuracy is 94% and AUC is 0.98

In [38]:
LRClassModel = class_balanced_model_fit(X_train_smote, y_train_smote, 
                                     X_test_smote, y_test_smote, 
                                     LogisticRegression(solver='lbfgs', max_iter=500),
                                     coef_show=1)

***** Accuracy of the classification model:  0.9398320895522388  *******

              precision    recall  f1-score   support

           0       0.93      0.96      0.94      5360
           1       0.96      0.92      0.94      5360

    accuracy                           0.94     10720
   macro avg       0.94      0.94      0.94     10720
weighted avg       0.94      0.94      0.94     10720


***** ROC_AUC score:  0.9818994173256849  *******
*************************************************

************ Top 10 positive features (variables) ************
      Word  Coefficient
     great    18.083379
      love    15.423238
      best    12.294290
      easy    11.063525
     clean    10.777082
      good    10.157003
    better     8.929209
     loved     8.734456
   perfect     8.238370
      nice     8.189582
 excellent     8.149311
   amazing     7.119679
   awesome     7.084521
   enjoyed     6.966062
  favorite     6.888610
 wonderful     6.344518
      free     6.114146
  

### Create a few sentences using the above positive and negative words and evaluate if they are classified correctly with Logistic Regression.

In [39]:
#Test some random text for classification - Do reviews classification as good or bad sentiment. 
Xfeatu = tfidf.transform(['i love this album it is very good', 
                          'an easy to use product',
                          'an awful product',
                          'they gave a free gift with the product',
                          'i went crazy after applying the lotion'])
print(LRClassModel.predict(Xfeatu))

[1 1 0 1 0]


### Use Multinomial Bayes as the classification model and evaluate.
#### Accuracy is 82% and AUC is 0.89

In [40]:
bayesClassModel = class_balanced_model_fit(X_train_smote, y_train_smote, 
                                     X_test_smote, y_test_smote, 
                                     MultinomialNB(),
                                     coef_show=1)

***** Accuracy of the classification model:  0.8095149253731343  *******

              precision    recall  f1-score   support

           0       0.84      0.76      0.80      5360
           1       0.78      0.86      0.82      5360

    accuracy                           0.81     10720
   macro avg       0.81      0.81      0.81     10720
weighted avg       0.81      0.81      0.81     10720


***** ROC_AUC score:  0.8917886952829137  *******
*************************************************

************ Top 10 positive features (variables) ************
      Word  Coefficient
     great    -3.930481
      love    -4.024045
    review    -4.126574
 promotion    -4.141571
 collected    -4.141803
     movie    -4.171696
       use    -4.215762
   product    -4.304792
      good    -4.401287
     clean    -4.423690
     wipes    -4.432484
    clorox    -4.588010
      easy    -4.686138
      like    -4.873915
      hair    -4.968508
    really    -5.049112
     smell    -5.086968
  

C:\Users\kumaraguru\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute coef_ was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


### Create a few sentences using the above positive and negative words and evaluate if they are classified correctly with Multinomial Bayes.

In [41]:
Xfeatu = tfidf.transform(['i love this album it is very good', 
                          'an easy to use product',
                          'an awful product',
                          'they gave a free gift with the product',
                          'i went crazy after applying the lotion'])
bayesClassModel.predict(Xfeatu)

array([1, 1, 0, 1, 0], dtype=int64)

### Use Random Forest as the classification model and evaluate.
### We use hyper-parameters like number of estimators, tree depth, maximum features to get a good accuracy.
### With a low value of estimators and lesser depth, the accuracy is poor.

In [42]:
from sklearn.ensemble import RandomForestClassifier

#### Estimators = 100, depth = 5
#### Accuracy is 81% and AUC is 0.88

In [43]:
randForest = RandomForestClassifier(n_estimators=100, max_depth=5, max_features=500, random_state=103, oob_score=True)
RFClassModel = class_balanced_model_fit(X_train_smote, y_train_smote, 
                                                 X_test_smote, y_test_smote, 
                                                 randForest,
                                                 coef_show=0)

***** Accuracy of the classification model:  0.8096082089552239  *******

              precision    recall  f1-score   support

           0       0.75      0.93      0.83      5360
           1       0.91      0.69      0.78      5360

    accuracy                           0.81     10720
   macro avg       0.83      0.81      0.81     10720
weighted avg       0.83      0.81      0.81     10720


***** ROC_AUC score:  0.8862818138783694  *******
*************************************************


#### Estimators = 500, depth = 20
#### Accuracy is 88% and AUC is 0.94

In [44]:
randForest = RandomForestClassifier(n_estimators=500, max_depth=20, max_features=500, random_state=103, oob_score=True)
RFClassModel = class_balanced_model_fit(X_train_smote, y_train_smote, 
                                                 X_test_smote, y_test_smote, 
                                                 randForest,
                                                 coef_show=0)

***** Accuracy of the classification model:  0.8868470149253731  *******

              precision    recall  f1-score   support

           0       0.86      0.93      0.89      5360
           1       0.92      0.85      0.88      5360

    accuracy                           0.89     10720
   macro avg       0.89      0.89      0.89     10720
weighted avg       0.89      0.89      0.89     10720


***** ROC_AUC score:  0.9434889451993764  *******
*************************************************


### Create a few sentences using the above positive and negative words and evaluate if they are classified correctly with Random Forest model.

In [45]:
Xfeatu = tfidf.transform(['i love this album it is very good', 
                          'an easy to use product',
                          'an awful product',
                          'they gave a free gift with the product',
                          'i went crazy after applying the lotion'])
RFClassModel.predict(Xfeatu)

array([1, 1, 0, 1, 0], dtype=int64)

### Of the 3 models, Logistic Regression yields a good balance of high accuracy and AUC. We will use Logistic Regression for classification of reviews text.

## Task 5: Build the Recommendation System. I have adopted the Upgrad Recommendation System module code for this section.

### Task 5 - Part A - User-based collaborative recommendation system

In [46]:
reviews_df

,name,reviews_rating,reviews_username,user_sentiment,reviews_tokenized
0,Pink Friday: Roman Reloaded Re-Up (w/dvd),5,joshua,Positive,i love this album it is very good more to the ...
1,Lundberg Organic Cinnamon Toast Rice Cakes,5,dorothy w,Positive,good flavor this review was collected as part ...
2,Lundberg Organic Cinnamon Toast Rice Cakes,5,dorothy w,Positive,good flavor
3,K-Y Love Sensuality Pleasure Gel,1,rebecca,Negative,i read through the reviews on here before look...
4,K-Y Love Sensuality Pleasure Gel,1,walker557,Negative,my husband bought this gel for us the gel caus...
...,...,...,...,...,...
29995,L'or233al Paris Elvive Extraordinary Clay Reba...,5,laurasnchz,Positive,i got this conditioner with influenster to try...
29996,L'or233al Paris Elvive Extraordinary Clay Reba...,5,scarlepadilla,Positive,i love it i received this for review purposes ...
29997,L'or233al Paris Elvive Extraordinary Clay Reba...,5,liviasuexo,Positive,first of all i love the smell of this product ...
29998,L'or233al Paris Elvive Extraordinary Clay Reba...,5,ktreed95,Positive,i received this through influenster and will n...


In [47]:
#reviews_df = reviews_df[['reviews_username', 'name', 'reviews_rating']]
#reviews_df.head()

In [48]:
reviews_df.reviews_rating.unique()

array([5, 1, 3, 4, 2], dtype=int64)

In [49]:
#Number of users
lst = reviews_df.reviews_username.value_counts()
print(lst)
print(len(lst))
print(sum(lst))

byamazon customer    41
mike                 41
chris                32
lisa                 16
sandy                15
                     ..
cecy1004              1
redbarren1            1
squall                1
okcmommy              1
kailey                1
Name: reviews_username, Length: 24914, dtype: int64
24914
29937


In [50]:
reviews_df.reviews_rating.value_counts()

5    20831
4     6020
1     1384
3     1345
2      420
Name: reviews_rating, dtype: int64

### Dividing the dataset into train and test

In [51]:
# Test and Train split of the dataset.
from sklearn.model_selection import train_test_split
train, test = train_test_split(reviews_df, test_size=0.30, random_state=31)

In [52]:
print(train.shape)
print(test.shape)

(21000, 5)
(9000, 5)


### I observed that the reviews records are repeated and this will cause issues with creation of pivot table. If I could find a means to average the ratings when creating the pivot, it would be good. Instead of pivot, we use a pivot_table that naturally averages out the repeated ratings. As an experiment, I created an excel sheet with 2 users anad 3 products and the reviews for brush are repeated by 'muthuraj'. You will see that pivot_table averages out the rating to 3.333.

In [53]:
#Proof that the default aggregate function for duplicates is mean
d = pd.read_csv('TestFile.csv' , encoding='latin-1')
print(d)
t = d.pivot_table(index='reviews_username', columns='name', values='reviews_rating')
t

   id   name  reviews_rating reviews_username
0   1  brush               3         muthuraj
1   2  brush               4         muthuraj
2   3    pen               5         muthuraj
3   4    pen               5            kumar
4   5  paint               4         muthuraj
5   6  paint               2            kumar
6   7  board               1         muthuraj
7   8  board               2            kumar
8   9  brush               3         muthuraj


name,board,brush,paint,pen
reviews_username,,,,
kumar,2.0,NaN,2.0,5.0
muthuraj,1.0,3.333333,4.0,5.0


In [54]:
print(len(reviews_df.reviews_username.unique()))
print(len(reviews_df.name.unique()))

tmp = reviews_df.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(0)
tmp.head()

24915
271


name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz","42 Dual Drop Leaf Table with 2 Madrid Chairs""",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),...,Walkers Stem Ginger Shortbread,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Wilton Black Dots Standard Baking Cups,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02dakota,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02deuce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
# Pivot the train ratings' dataset into matrix format in which columns are items and the rows are user.
df_pivot = train.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(0)
df_pivot.head(5)

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Creating dummy train & dummy test dataset
These dataset will be used for prediction 
- Dummy train will be used later for prediction of the movies which has not been rated by the user. To ignore the movies rated by the user, we will mark it as 0 during prediction. The movies not rated by user is marked as 1 for prediction in dummy train dataset. 

- Dummy test will be used for evaluation. To evaluate, we will only make prediction on the movies rated by the user. So, this is marked as 1. This is just opposite of dummy_train.

In [56]:
# Copy the train dataset into dummy_train
dummy_train = train.copy()
dummy_train.head()

,name,reviews_rating,reviews_username,user_sentiment,reviews_tokenized
11155,Clorox Disinfecting Wipes Value Pack Scented 1...,5,gor05751,Positive,it is quick and easy compared to using a spray...
22114,Vaseline Intensive Care Healthy Hands Stronger...,4,annavsl,Positive,stronger nails after a week of daily use not t...
26218,"Finish Gelpacs Dishwasher Detergent, Orange Sc...",5,bluenose,Positive,really does clean everything and so good on gl...
18966,Clorox Disinfecting Bathroom Cleaner,5,kiki,Positive,these wipes are great for cleaning in the kitc...
29999,L'or233al Paris Elvive Extraordinary Clay Reba...,5,kcoopxoxo,Positive,i received this product complimentary from inf...


In [57]:
print(dummy_train['reviews_rating'].value_counts())
print(dummy_train.shape)

5    14514
4     4234
1      995
3      957
2      300
Name: reviews_rating, dtype: int64
(21000, 5)


In [58]:
# The movies not rated by user is marked as 1 for prediction. 
dummy_train['reviews_rating'] = dummy_train['reviews_rating'].apply(lambda x: 0 if x>=1 else 1)

In [59]:
dummy_train.reviews_rating.unique()

array([0], dtype=int64)

In [60]:
dummy_train.head()

,name,reviews_rating,reviews_username,user_sentiment,reviews_tokenized
11155,Clorox Disinfecting Wipes Value Pack Scented 1...,0,gor05751,Positive,it is quick and easy compared to using a spray...
22114,Vaseline Intensive Care Healthy Hands Stronger...,0,annavsl,Positive,stronger nails after a week of daily use not t...
26218,"Finish Gelpacs Dishwasher Detergent, Orange Sc...",0,bluenose,Positive,really does clean everything and so good on gl...
18966,Clorox Disinfecting Bathroom Cleaner,0,kiki,Positive,these wipes are great for cleaning in the kitc...
29999,L'or233al Paris Elvive Extraordinary Clay Reba...,0,kcoopxoxo,Positive,i received this product complimentary from inf...


In [61]:
# Convert the dummy train dataset into matrix format.
dummy_train = dummy_train.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(1)

In [62]:
dummy_train.shape

(18252, 252)

#### Checking for a given user if the reviewed items are set to 0 and rest to 1. All good so far.

In [63]:
print(dummy_train.loc['abby'].value_counts())
print(dummy_train.loc['abismomy'].value_counts())
print(dummy_train.loc['ac94'].value_counts())


1.0    249
0.0      3
Name: abby, dtype: int64
1.0    250
0.0      2
Name: abismomy, dtype: int64
1.0    250
0.0      2
Name: ac94, dtype: int64


In [64]:
dummy_train

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
00sab00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
01impala,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0325home,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
06stidriver,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zulaa118,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
zxcsdfd,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
zxjki,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


**Cosine Similarity**

Cosine Similarity is a measurement that quantifies the similarity between two vectors [Which is Rating Vector in this case] 

**Adjusted Cosine**

Adjusted cosine similarity is a modified version of vector-based similarity where we incorporate the fact that different users have different ratings schemes. In other words, some users might rate items highly in general, and others might give items lower ratings as a preference. To handle this nature from rating given by user , we subtract average ratings for each user from each user's rating for different movies.

### User Similarity Matrix

### Using Cosine Similarity
#### Pair-wise similarity is done row-wise in the pivot table

In [65]:
from sklearn.metrics.pairwise import pairwise_distances

# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_pivot, metric='cosine')
print(user_correlation)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


#### Since the above matrix looks suspicious, confirm that the correlation values are set to meaningful values.

In [66]:
print(sum(user_correlation[0])/len(user_correlation[0]))
print(sum(user_correlation[1])/len(user_correlation[1]))
print(sum(user_correlation[2])/len(user_correlation[2]))
print(sum(user_correlation[3])/len(user_correlation[3]))
print(sum(user_correlation[4])/len(user_correlation[4]))
print(sum(user_correlation[5])/len(user_correlation[5]))
print(sum(user_correlation[6])/len(user_correlation[6]))
print(sum(user_correlation[7])/len(user_correlation[7]))


0.0007699409750198169
0.025349981632427198
0.11795404456236924
0.2965389464248795
0.2965389464248795
0.06681913443692394
0.02378234824574021
0.003206839857539405


In [67]:
user_correlation[np.isnan(user_correlation)] = 0

In [68]:
print(user_correlation)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [69]:
user_correlation.shape

(18252, 18252)

### Using adjusted Cosine 

### Here, we are not removing the NaN values and calculating the mean only for the items rated by the user

In [70]:
# Create a user-item review matrix.
df_pivot = train.pivot_table(index='reviews_username', columns='name', values='reviews_rating')

In [71]:
df_pivot.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01impala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0325home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06stidriver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Normalising the rating of the item for each user around 0 mean

In [72]:
#This does the mean for each row as the axis is 1
mean = np.nanmean(df_pivot, axis=1)

In [73]:
print(mean)
print(len(mean))

[4. 1. 3. ... 2. 5. 5.]
18252


In [74]:
df_subtracted = (df_pivot.T - mean).T

In [75]:
df_subtracted.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01impala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0325home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06stidriver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Finding cosine similarity

In [76]:
from sklearn.metrics.pairwise import pairwise_distances

In [77]:
# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Prediction - User User

Doing the prediction for the users which are positively related with other users, and not the users which are negatively related as we are interested in the users which are more similar to the current users. So, ignoring the correlation for values less than 0. 

In [78]:
user_correlation[user_correlation<0]=0
user_correlation

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [79]:
#Check if there are legitimate values in the correlation matrix
sum(sum(user_correlation))

10185.884430507153

Rating predicted by the user (for items rated as well as not rated) is the weighted sum of correlation with the item rating (as present in the rating dataset). 

In [80]:
user_predicted_ratings = np.dot(user_correlation, df_pivot.fillna(0))
user_predicted_ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [81]:
user_predicted_ratings.shape

(18252, 252)

In [82]:
sum(sum(user_predicted_ratings))

120324.65698632221

Since we are interested only in the items not rated by the user, we will ignore the items rated by the user by making it zero. 

In [83]:
user_final_rating = np.multiply(user_predicted_ratings,dummy_train)
user_final_rating.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Check for our algorithm correctness - Sum the numbers by items to see if anything meaningful is arrived at. For these items, check in the excel sheet and pick a user (who is in the train data set) and get recommendations for her.


In [82]:
print(user_final_rating.T.sum(axis=1).sort_values(ascending=False)[0:60])

name
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)                                        5404.701432
The Resident Evil Collection 5 Discs (blu-Ray)                                                             5196.472401
Mike Dave Need Wedding Dates (dvd + Digital)                                                               5117.180772
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd                                            4756.145677
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                                                       4343.492836
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                                                  4053.226287
Red (special Edition) (dvdvideo)                                                                           2982.062401
Clorox Disinfecting Bathroom Cleaner                                                                       2413.224154
Windex Original Glass Cleaner Refill 67.6oz

In [83]:
# This block will execute only if 'user' is in the training dataset.
d = user_final_rating.loc['abby'].sort_values(ascending=False)[0:10]
d

name
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd        6.269380
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)    5.679494
Clorox Disinfecting Bathroom Cleaner                                   3.651637
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                   3.370093
The Resident Evil Collection 5 Discs (blu-Ray)                         3.370093
Tostitos Bite Size Tortilla Chips                                      3.370093
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total              3.370093
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo               3.370093
Chester's Cheese Flavored Puffcorn Snacks                              3.333333
Tostitos Simply Blue Corn Tortilla Chips                               3.333333
Name: abby, dtype: float64

### Finding the top 20 recommendations for the *user*

In [84]:
#Take the user name as input.
user_input = input("Enter user name ")
print(user_input)

Enter user name abby
abby


In [85]:
user_final_rating.head(2)

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
top20UserBased = user_final_rating.loc[user_input].sort_values(ascending=False)[0:20]
top20UserBased

name
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd                                  6.269380
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)                              5.679494
Clorox Disinfecting Bathroom Cleaner                                                             3.651637
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                                             3.370093
The Resident Evil Collection 5 Discs (blu-Ray)                                                   3.370093
Tostitos Bite Size Tortilla Chips                                                                3.370093
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                                        3.370093
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo                                         3.370093
Chester's Cheese Flavored Puffcorn Snacks                                                        3.333333
Tostitos Simply Blue Corn Tortilla Chips 

### Evaluation - User User 

Evaluation will be same as you have seen above for the prediction. The only difference being, you will evaluate for the item already rated by the user insead of predicting it for the item not rated by the user. 

In [87]:
# Find out the common users of test and train dataset.
common = test[test.reviews_username.isin(train.reviews_username)]
common.shape

(2049, 5)

In [88]:
common.head()

,name,reviews_rating,reviews_username,user_sentiment,reviews_tokenized
20634,Starbucks Iced Expresso Classics Vanilla Latte...,5,girlie,Positive,love it
26479,Aveeno Baby Continuous Protection Lotion Sunsc...,1,kasheville,Positive,my year old daughter has sensitive skin and af...
15098,Avery174 Ready Index Contemporary Table Of Con...,5,arlene,Positive,wow what a difference ease makes thanks for th...
17968,Vaseline Intensive Care Lip Therapy Cocoa Butter,5,mimi3,Positive,love this product its amazing made my lips sup...
28020,Planes: Fire Rescue (2 Discs) (includes Digita...,5,movielover,Positive,this movie is wonderful it has a great story l...


In [89]:
# convert into the user-item matrix.
common_user_based_matrix = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating')

In [90]:
common_user_based_matrix.head()

name,100:Complete First Season (blu-Ray),"42 Dual Drop Leaf Table with 2 Madrid Chairs""",Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz","Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz","Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter",Axe Dry Anti-Perspirant Deodorant Invisible Solid Phoenix,"BRIDGESTONE 130/70ZR18M/C(63W)FRONT EXEDRA G851, CRUISER RADL",...,Various - Country's Greatest Gospel:Gold Ed (cd),Various - Red Hot Blue:Tribute To Cole Porter (cd),Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234asdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123charlie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
# Convert the user_correlation matrix into dataframe.
user_correlation_df = pd.DataFrame(user_correlation)

In [92]:
user_correlation_df

,0,1,2,3,4,5,6,7,8,9,...,18242,18243,18244,18245,18246,18247,18248,18249,18250,18251
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
df_subtracted.head(1)

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
df_subtracted.index

Index(['00dog3', '00sab00', '01impala', '0325home', '06stidriver', '09mommy11',
       '1085', '10ten', '11111111aaaaaaaaaaaaaaaaa', '1143mom',
       ...
       'zozo0o', 'zpalma', 'zsazsa', 'zt313', 'zubb', 'zulaa118', 'zxcsdfd',
       'zxjki', 'zyiah4', 'zzdiane'],
      dtype='object', name='reviews_username', length=18252)

In [95]:
user_correlation_df['reviews_username'] = df_subtracted.index
user_correlation_df.set_index('reviews_username',inplace=True)
user_correlation_df.head()

,0,1,2,3,4,5,6,7,8,9,...,18242,18243,18244,18245,18246,18247,18248,18249,18250,18251
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
common.head(1)

,name,reviews_rating,reviews_username,user_sentiment,reviews_tokenized
20634,Starbucks Iced Expresso Classics Vanilla Latte...,5,girlie,Positive,love it


In [97]:
list_name = common.reviews_username.tolist()
user_correlation_df.columns = df_subtracted.index.tolist()
user_correlation_df_1 =  user_correlation_df[user_correlation_df.index.isin(list_name)]

In [98]:
len(list_name)

2049

In [99]:
user_correlation_df.columns

Index(['00dog3', '00sab00', '01impala', '0325home', '06stidriver', '09mommy11',
       '1085', '10ten', '11111111aaaaaaaaaaaaaaaaa', '1143mom',
       ...
       'zozo0o', 'zpalma', 'zsazsa', 'zt313', 'zubb', 'zulaa118', 'zxcsdfd',
       'zxjki', 'zyiah4', 'zzdiane'],
      dtype='object', length=18252)

In [100]:
user_correlation_df_1

,00dog3,00sab00,01impala,0325home,06stidriver,09mommy11,1085,10ten,11111111aaaaaaaaaaaaaaaaa,1143mom,...,zozo0o,zpalma,zsazsa,zt313,zubb,zulaa118,zxcsdfd,zxjki,zyiah4,zzdiane
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234asdf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123charlie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zach,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zapman,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zburt5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
user_correlation_df_1.shape

(1728, 18252)

In [102]:
user_correlation_df_2 = user_correlation_df_1.T[user_correlation_df_1.T.index.isin(list_name)]

In [103]:
user_correlation_df_3 = user_correlation_df_2.T

In [104]:
user_correlation_df_3.head()

,00sab00,1234,1234asdf,123charlie,1943,4cloroxl,4djs,7inthenest,85lisa,a hutch,...,ygtz,yoadrian,yohnie1,yummy,yvonne,zach,zapman,zburt5,zebras,zmom
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234asdf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123charlie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1943,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
user_correlation_df_3.shape

(1728, 1728)

In [106]:
user_correlation_df_3[user_correlation_df_3<0]=0

common_user_predicted_ratings = np.dot(user_correlation_df_3, common_user_based_matrix.fillna(0))
common_user_predicted_ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [107]:
dummy_test = common.copy()
dummy_test['reviews_rating'] = dummy_test['reviews_rating'].apply(lambda x: 1 if x>=1 else 0)
dummy_test = dummy_test.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(0)

In [108]:
dummy_test.shape

(1728, 126)

In [109]:
common_user_predicted_ratings = np.multiply(common_user_predicted_ratings,dummy_test)

In [110]:
common_user_predicted_ratings.head(2)

name,100:Complete First Season (blu-Ray),"42 Dual Drop Leaf Table with 2 Madrid Chairs""",Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz","Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz","Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter",Axe Dry Anti-Perspirant Deodorant Invisible Solid Phoenix,"BRIDGESTONE 130/70ZR18M/C(63W)FRONT EXEDRA G851, CRUISER RADL",...,Various - Country's Greatest Gospel:Gold Ed (cd),Various - Red Hot Blue:Tribute To Cole Porter (cd),Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Calculating the RMSE for only the items rated by user. For RMSE, normalising the rating to (1,5) range.

In [111]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_user_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

MinMaxScaler(feature_range=(1, 5))
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


C:\Users\kumaraguru\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\kumaraguru\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [112]:
y.shape

(1728, 126)

In [113]:
common_ = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating')

In [114]:
common_

name,100:Complete First Season (blu-Ray),"42 Dual Drop Leaf Table with 2 Madrid Chairs""",Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz","Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz","Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter",Axe Dry Anti-Perspirant Deodorant Invisible Solid Phoenix,"BRIDGESTONE 130/70ZR18M/C(63W)FRONT EXEDRA G851, CRUISER RADL",...,Various - Country's Greatest Gospel:Gold Ed (cd),Various - Red Hot Blue:Tribute To Cole Porter (cd),Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234asdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123charlie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zapman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
zburt5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [116]:
print(total_non_nan)

314


In [117]:
rmse = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse)

2.498437231297542


### Task 5 - Part B - Item-based collaborative recommendation system

### Item Based Similarity

Taking the transpose of the rating matrix to normalize the rating around the mean for different item. In the user based similarity, we had taken mean for each user instead of each item. 

In [118]:
df_pivot = train.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T
df_pivot.head()

reviews_username,00dog3,00sab00,01impala,0325home,06stidriver,09mommy11,1085,10ten,11111111aaaaaaaaaaaaaaaaa,1143mom,...,zozo0o,zpalma,zsazsa,zt313,zubb,zulaa118,zxcsdfd,zxjki,zyiah4,zzdiane
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100:Complete First Season (blu-Ray),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2x Ultra Era with Oxi Booster, 50fl oz",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Normalising the movie rating for each movie for using the Adujsted Cosine

In [119]:
mean = np.nanmean(df_pivot, axis=1)
df_subtracted = (df_pivot.T-mean).T

In [120]:
df_subtracted.head()

reviews_username,00dog3,00sab00,01impala,0325home,06stidriver,09mommy11,1085,10ten,11111111aaaaaaaaaaaaaaaaa,1143mom,...,zozo0o,zpalma,zsazsa,zt313,zubb,zulaa118,zxcsdfd,zxjki,zyiah4,zzdiane
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100:Complete First Season (blu-Ray),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2x Ultra Era with Oxi Booster, 50fl oz",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finding the cosine similarity using pairwise distances approach

In [121]:
from sklearn.metrics.pairwise import pairwise_distances

In [122]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation)

[[ 1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          1.          0.         ... -0.00657044  0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.         -0.00657044  0.         ...  1.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          1.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   1.        ]]


Filtering the correlation only for which the value is greater than 0. (Positively correlated)

In [123]:
item_correlation[item_correlation<0]=0
item_correlation

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

# Prediction - Item Item

In [124]:
item_predicted_ratings = np.dot((df_pivot.fillna(0).T),item_correlation)
item_predicted_ratings

array([[0.        , 0.        , 0.        , ..., 0.02352264, 0.        ,
        0.        ],
       [0.        , 0.00117139, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04848007, 0.        , ..., 0.        , 0.        ,
        0.00338607],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00062229, 0.00063779,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00155571, 0.00159447,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.03252811, 0.02124867,
        0.        ]])

In [125]:
item_predicted_ratings.shape

(18252, 252)

In [126]:
dummy_train.shape

(18252, 252)

### Filtering the rating only for the items not rated by the user for recommendation

In [127]:
item_final_rating = np.multiply(item_predicted_ratings,dummy_train)
item_final_rating.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.023523,0.000000,0.000000
00sab00,0.0,0.001171,0.0,0.0,0.0,0.0,0.0,0.0,0.003316,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
01impala,0.0,0.048480,0.0,0.0,0.0,0.0,0.0,0.0,0.014725,0.000000,...,0.0,0.0,0.0,0.006729,0.002859,0.0,0.0,0.000000,0.000000,0.003386
0325home,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
06stidriver,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000


### Finding the top 20 recommendation for the *user*

In [128]:
# Take the user ID as input
user_input = input("Enter user name")
print(user_input)

Enter user nameabby
abby


In [129]:
# Recommending the Top 5 products to the user.
top20ItemBased = item_final_rating.loc[user_input].sort_values(ascending=False)[0:20]
top20ItemBased

name
Chester's Cheese Flavored Puffcorn Snacks                                         0.546107
Windex Original Glass Cleaner Refill 67.6oz (2 Liter)                             0.167279
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo                          0.086751
Pleasant Hearth 1,800 sq ft Wood Burning Stove with Blower, Medium, LWS-127201    0.042787
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)               0.032291
Nexxus Exxtra Gel Style Creation Sculptor                                         0.030927
Aussie Aussome Volume Shampoo, 13.5 Oz                                            0.028226
Mike Dave Need Wedding Dates (dvd + Digital)                                      0.027196
Diet Canada Dry Ginger Ale - 12pk/12 Fl Oz Cans                                   0.023710
Dark Shadows (includes Digital Copy) (ultraviolet) (dvdvideo)                     0.021194
Cheetos Crunchy Flamin' Hot Cheese Flavored Snacks                                0.0

# Evaluation - Item Item

Evaluation will we same as you have seen above for the prediction. The only difference being, you will evaluate for the movie already rated by the user insead of predicting it for the movie not rated by the user. 

In [130]:
test.columns

Index(['name', 'reviews_rating', 'reviews_username', 'user_sentiment',
       'reviews_tokenized'],
      dtype='object')

In [131]:
common =  test[test.name.isin(train.name)]
common.shape

(8978, 5)

In [132]:
common.head(4)

,name,reviews_rating,reviews_username,user_sentiment,reviews_tokenized
4332,Hoover174 Platinum Collection153 Lightweight B...,5,sansname,Positive,i have only owned this product for weeks so co...
3434,Pantene Pro-V Expert Collection Age Defy Condi...,5,jennypstrong,Positive,this is thick stuff i was surprised by the con...
20634,Starbucks Iced Expresso Classics Vanilla Latte...,5,girlie,Positive,love it
26479,Aveeno Baby Continuous Protection Lotion Sunsc...,1,kasheville,Positive,my year old daughter has sensitive skin and af...


In [133]:
common_item_based_matrix = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T

In [134]:
common_item_based_matrix.shape

(199, 8371)

In [135]:
item_correlation_df = pd.DataFrame(item_correlation)

In [136]:
item_correlation_df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
item_correlation_df['name'] = df_subtracted.index
item_correlation_df.set_index('name',inplace=True)
item_correlation_df.head()

,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100:Complete First Season (blu-Ray),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"2x Ultra Era with Oxi Booster, 50fl oz",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [138]:
list_name = common.name.tolist()

In [139]:
item_correlation_df.columns = df_subtracted.index.tolist()
item_correlation_df_1 =  item_correlation_df[item_correlation_df.index.isin(list_name)]

In [140]:
item_correlation_df_2 = item_correlation_df_1.T[item_correlation_df_1.T.index.isin(list_name)]
item_correlation_df_3 = item_correlation_df_2.T

In [141]:
item_correlation_df_3.head()

,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),4C Grated Parmesan Cheese 100% Natural 8oz Shaker,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Annie's Homegrown Gluten Free Double Chocolate Chip Granola Bars,Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz",...,Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100:Complete First Season (blu-Ray),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Africa's Best No-Lye Dual Conditioning Relaxer System Super,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
item_correlation_df_3[item_correlation_df_3<0]=0

common_item_predicted_ratings = np.dot(item_correlation_df_3, common_item_based_matrix.fillna(0))
common_item_predicted_ratings

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.08080011, 0.06464009, ..., 0.01395515, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.01951687, 0.        , 0.        , ..., 0.        , 0.00155571,
        0.00124457],
       [0.0127492 , 0.        , 0.        , ..., 0.        , 0.00159447,
        0.00127557],
       [0.        , 0.00564345, 0.00451476, ..., 0.        , 0.        ,
        0.        ]])

In [143]:
common_item_predicted_ratings.shape

(199, 8371)

Dummy test will be used for evaluation. To evaluate, we will only make prediction on the items rated by the user. So, this is marked as 1. This is just opposite of dummy_train.

In [144]:
dummy_test = common.copy()
dummy_test['reviews_rating'] = dummy_test['reviews_rating'].apply(lambda x: 1 if x>=1 else 0)
dummy_test = dummy_test.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T.fillna(0)
common_item_predicted_ratings = np.multiply(common_item_predicted_ratings,dummy_test)

The products not rated is marked as 0 for evaluation. And make the item- item matrix representaion.

In [145]:
common_ = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T

In [146]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_item_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

MinMaxScaler(feature_range=(1, 5))
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


C:\Users\kumaraguru\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\kumaraguru\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [147]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [148]:
rmse = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse)

3.559992328750575


## Task 6: Recommendation of Top 20 Products to a specified user. 
### Since user-based collaborative system has a lesser RMSE, we go with that. 

In [149]:
train

,name,reviews_rating,reviews_username,user_sentiment,reviews_tokenized
11155,Clorox Disinfecting Wipes Value Pack Scented 1...,5,gor05751,Positive,it is quick and easy compared to using a spray...
22114,Vaseline Intensive Care Healthy Hands Stronger...,4,annavsl,Positive,stronger nails after a week of daily use not t...
26218,"Finish Gelpacs Dishwasher Detergent, Orange Sc...",5,bluenose,Positive,really does clean everything and so good on gl...
18966,Clorox Disinfecting Bathroom Cleaner,5,kiki,Positive,these wipes are great for cleaning in the kitc...
29999,L'or233al Paris Elvive Extraordinary Clay Reba...,5,kcoopxoxo,Positive,i received this product complimentary from inf...
...,...,...,...,...,...
3735,Diet Canada Dry Ginger Ale - 12pk/12 Fl Oz Cans,5,mary,Positive,my husband had a heartattack in and this is wh...
14946,Clorox Disinfecting Wipes Value Pack Scented 1...,5,mwah25,Positive,these are the best wipes out there they clean ...
27518,Planes: Fire Rescue (2 Discs) (includes Digita...,4,lilym,Positive,we were a bit excited to see this movie after ...
26640,Aveeno Baby Continuous Protection Lotion Sunsc...,5,kim,Positive,i am honestly really surprised to see that the...


In [150]:
test

,name,reviews_rating,reviews_username,user_sentiment,reviews_tokenized
4332,Hoover174 Platinum Collection153 Lightweight B...,5,sansname,Positive,i have only owned this product for weeks so co...
3434,Pantene Pro-V Expert Collection Age Defy Condi...,5,jennypstrong,Positive,this is thick stuff i was surprised by the con...
20634,Starbucks Iced Expresso Classics Vanilla Latte...,5,girlie,Positive,love it
26479,Aveeno Baby Continuous Protection Lotion Sunsc...,1,kasheville,Positive,my year old daughter has sensitive skin and af...
29726,L'or233al Paris Elvive Extraordinary Clay Reba...,5,payy,Positive,i thought this was fantastic my hair felt supe...
...,...,...,...,...,...
28375,Various Artists - Choo Choo Soul (cd),5,bala,Positive,our month old granddaughter loves it the songs...
20889,Tostitos Bite Size Tortilla Chips,2,ronald,Negative,chips wereold and had very little taste
3505,Chex Muddy Buddies Brownie Supreme Snack Mix,5,elunsford,Positive,really good good chocolate taste these are add...
11361,Clorox Disinfecting Wipes Value Pack Scented 1...,5,meli23melo,Positive,very good product i would recommended to all o...


In [151]:
item_final_rating

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.023523,0.000000,0.000000
00sab00,0.0,0.001171,0.0,0.0,0.0,0.0,0.0,0.0,0.003316,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
01impala,0.0,0.048480,0.0,0.0,0.0,0.0,0.0,0.0,0.014725,0.000000,...,0.0,0.0,0.0,0.006729,0.002859,0.0,0.0,0.000000,0.000000,0.003386
0325home,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
06stidriver,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zulaa118,0.0,0.013955,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.039908,0.000000,0.0,0.0,0.000000,0.000000,0.000000
zxcsdfd,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
zxjki,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000534,0.001702,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000622,0.000638,0.000000


In [152]:
top20UserBased

name
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd                                  6.269380
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)                              5.679494
Clorox Disinfecting Bathroom Cleaner                                                             3.651637
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                                             3.370093
The Resident Evil Collection 5 Discs (blu-Ray)                                                   3.370093
Tostitos Bite Size Tortilla Chips                                                                3.370093
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                                        3.370093
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo                                         3.370093
Chester's Cheese Flavored Puffcorn Snacks                                                        3.333333
Tostitos Simply Blue Corn Tortilla Chips 

## Task 7: Fine-Tuning the Recommendation System and Recommendation of Top 5 Products.

### Print our old tokenized reviews dataframe

In [153]:
print(reviews_df)

                                                    name  reviews_rating  \
0              Pink Friday: Roman Reloaded Re-Up (w/dvd)               5   
1             Lundberg Organic Cinnamon Toast Rice Cakes               5   
2             Lundberg Organic Cinnamon Toast Rice Cakes               5   
3                       K-Y Love Sensuality Pleasure Gel               1   
4                       K-Y Love Sensuality Pleasure Gel               1   
...                                                  ...             ...   
29995  L'or233al Paris Elvive Extraordinary Clay Reba...               5   
29996  L'or233al Paris Elvive Extraordinary Clay Reba...               5   
29997  L'or233al Paris Elvive Extraordinary Clay Reba...               5   
29998  L'or233al Paris Elvive Extraordinary Clay Reba...               5   
29999  L'or233al Paris Elvive Extraordinary Clay Reba...               5   

      reviews_username user_sentiment  \
0               joshua       Positive   
1    

### Identify the sentiments of the top 20 items, average them per item and pick top 5. 

#### TF-IDF Vectorize all the reviews and classify them as positive or negative and merge them with the dataframe.

In [154]:
reviews_df

,name,reviews_rating,reviews_username,user_sentiment,reviews_tokenized
0,Pink Friday: Roman Reloaded Re-Up (w/dvd),5,joshua,Positive,i love this album it is very good more to the ...
1,Lundberg Organic Cinnamon Toast Rice Cakes,5,dorothy w,Positive,good flavor this review was collected as part ...
2,Lundberg Organic Cinnamon Toast Rice Cakes,5,dorothy w,Positive,good flavor
3,K-Y Love Sensuality Pleasure Gel,1,rebecca,Negative,i read through the reviews on here before look...
4,K-Y Love Sensuality Pleasure Gel,1,walker557,Negative,my husband bought this gel for us the gel caus...
...,...,...,...,...,...
29995,L'or233al Paris Elvive Extraordinary Clay Reba...,5,laurasnchz,Positive,i got this conditioner with influenster to try...
29996,L'or233al Paris Elvive Extraordinary Clay Reba...,5,scarlepadilla,Positive,i love it i received this for review purposes ...
29997,L'or233al Paris Elvive Extraordinary Clay Reba...,5,liviasuexo,Positive,first of all i love the smell of this product ...
29998,L'or233al Paris Elvive Extraordinary Clay Reba...,5,ktreed95,Positive,i received this through influenster and will n...


In [155]:
#Do reviews classification as good or bad sentiment. 
tfidfFeatures = tfidf.transform(reviews_df.reviews_tokenized)
sentcls = LRClassModel.predict(tfidfFeatures)

In [156]:
#Merge the class to the dataframe
sntmtClassSeries = pd.Series(sentcls, name = "sentiment_class")
reviews_df = reviews_df.join(sntmtClassSeries)
print(reviews_df[['name', 'reviews_tokenized', 'sentiment_class']])

                                                    name  \
0              Pink Friday: Roman Reloaded Re-Up (w/dvd)   
1             Lundberg Organic Cinnamon Toast Rice Cakes   
2             Lundberg Organic Cinnamon Toast Rice Cakes   
3                       K-Y Love Sensuality Pleasure Gel   
4                       K-Y Love Sensuality Pleasure Gel   
...                                                  ...   
29995  L'or233al Paris Elvive Extraordinary Clay Reba...   
29996  L'or233al Paris Elvive Extraordinary Clay Reba...   
29997  L'or233al Paris Elvive Extraordinary Clay Reba...   
29998  L'or233al Paris Elvive Extraordinary Clay Reba...   
29999  L'or233al Paris Elvive Extraordinary Clay Reba...   

                                       reviews_tokenized  sentiment_class  
0      i love this album it is very good more to the ...                1  
1      good flavor this review was collected as part ...                1  
2                                            good f

#### Group the sentiments by item name and average them.

In [157]:
groupedDf = reviews_df.groupby(['name'])
product_class = groupedDf['sentiment_class'].agg(mean_class=np.mean)

In [158]:
print(type(product_class))
print(product_class)

<class 'pandas.core.frame.DataFrame'>
<bound method DataFrame.info of                                                     mean_class
name                                                          
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. F...    0.666667
100:Complete First Season (blu-Ray)                   0.877698
2017-2018 Brownline174 Duraflex 14-Month Planne...    0.000000
2x Ultra Era with Oxi Booster, 50fl oz                1.000000
42 Dual Drop Leaf Table with 2 Madrid Chairs"         1.000000
...                                                        ...
Weleda Everon Lip Balm                                1.000000
Wilton Black Dots Standard Baking Cups                1.000000
Windex Original Glass Cleaner Refill 67.6oz (2 ...    0.744253
Yes To Carrots Nourishing Body Wash                   0.969231
Yes To Grapefruit Rejuvenating Body Wash              0.769231

[271 rows x 1 columns]>


In [166]:
# Save the product class.
import joblib


# Save the product class as a pickle in a file. 
joblib.dump(product_class, 'product_class.pkl') 

# Load the model from the file 
product_class_from_pkl = joblib.load('product_class.pkl')
print(product_class_from_pkl)

                                                    mean_class
name                                                          
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. F...    0.666667
100:Complete First Season (blu-Ray)                   0.877698
2017-2018 Brownline174 Duraflex 14-Month Planne...    0.000000
2x Ultra Era with Oxi Booster, 50fl oz                1.000000
42 Dual Drop Leaf Table with 2 Madrid Chairs"         1.000000
...                                                        ...
Weleda Everon Lip Balm                                1.000000
Wilton Black Dots Standard Baking Cups                1.000000
Windex Original Glass Cleaner Refill 67.6oz (2 ...    0.744253
Yes To Carrots Nourishing Body Wash                   0.969231
Yes To Grapefruit Rejuvenating Body Wash              0.769231

[271 rows x 1 columns]


In [159]:
product_class.index

Index(['0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest',
       '100:Complete First Season (blu-Ray)',
       '2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black',
       '2x Ultra Era with Oxi Booster, 50fl oz',
       '42 Dual Drop Leaf Table with 2 Madrid Chairs"',
       '4C Grated Parmesan Cheese 100% Natural 8oz Shaker',
       '5302050 15/16 FCT/HOSE ADAPTOR',
       'Africa's Best No-Lye Dual Conditioning Relaxer System Super',
       'Alberto VO5 Salon Series Smooth Plus Sleek Shampoo',
       'Alex Cross (dvdvideo)',
       ...
       'Walkers Stem Ginger Shortbread',
       'Wallmount Server Cabinet (450mm, 9 RU)',
       'Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee',
       'WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black',
       'Wedding Wishes Wedding Guest Book', 'Weleda Everon Lip Balm',
       'Wilton Black Dots Standard Baking Cups',
       'Windex Original 

In [160]:
for x in product_class.index:
    print(product_class.loc[x])

mean_class    0.666667
Name: 0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest, dtype: float64
mean_class    0.877698
Name: 100:Complete First Season (blu-Ray), dtype: float64
mean_class    0.0
Name: 2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black, dtype: float64
mean_class    1.0
Name: 2x Ultra Era with Oxi Booster, 50fl oz, dtype: float64
mean_class    1.0
Name: 42 Dual Drop Leaf Table with 2 Madrid Chairs", dtype: float64
mean_class    0.666667
Name: 4C Grated Parmesan Cheese 100% Natural 8oz Shaker, dtype: float64
mean_class    0.0
Name: 5302050 15/16 FCT/HOSE ADAPTOR, dtype: float64
mean_class    0.833333
Name: Africa's Best No-Lye Dual Conditioning Relaxer System Super, dtype: float64
mean_class    0.666667
Name: Alberto VO5 Salon Series Smooth Plus Sleek Shampoo, dtype: float64
mean_class    0.801418
Name: Alex Cross (dvdvideo), dtype: float64
mean_class    0.454545
Name: All,bran Complete Wheat Flakes, 18 Oz., dtype: float64
mean_class    1.0
Name:

Name: Maybelline Clean Express! Makeup Removing Lotion, dtype: float64
mean_class    0.5
Name: Meguiar's Deep Crystal Car Wash 64-Oz., dtype: float64
mean_class    0.965517
Name: Meguiar's Ultimate Quik Detailer 22-Oz., dtype: float64
mean_class    1.0
Name: Meguiars Interior Detailer Wipes 25ct, dtype: float64
mean_class    0.769231
Name: Metzeler ME888 Marathon Ultra Custom Touring Front Tire 100/90-19, dtype: float64
mean_class    1.0
Name: Mia Shower Curtain Green/brown - Lush Decor174, dtype: float64
mean_class    0.797886
Name: Mike Dave Need Wedding Dates (dvd + Digital), dtype: float64
mean_class    0.9
Name: Mill Creek Aloe Vera & Paba Lotion, dtype: float64
mean_class    0.6
Name: Moom Express Pre Waxed Strips For Face & Bikini, dtype: float64
mean_class    0.866667
Name: Mrs. Meyer's Lemon Verbena Surface Scrub, dtype: float64
mean_class    0.965517
Name: Mrs. Meyer's174 Lemon Verbena Laundry Scent Booster - 18oz, dtype: float64
mean_class    0.571429
Name: Musselman Apple S

Name: Wallmount Server Cabinet (450mm, 9 RU), dtype: float64
mean_class    0.818182
Name: Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee, dtype: float64
mean_class    0.75
Name: WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black, dtype: float64
mean_class    1.0
Name: Wedding Wishes Wedding Guest Book, dtype: float64
mean_class    1.0
Name: Weleda Everon Lip Balm, dtype: float64
mean_class    1.0
Name: Wilton Black Dots Standard Baking Cups, dtype: float64
mean_class    0.744253
Name: Windex Original Glass Cleaner Refill 67.6oz (2 Liter), dtype: float64
mean_class    0.969231
Name: Yes To Carrots Nourishing Body Wash, dtype: float64
mean_class    0.769231
Name: Yes To Grapefruit Rejuvenating Body Wash, dtype: float64


In [161]:
# Take the item name as input and print its average sentiment.
item_name = input("Item name ").strip()
print(item_name)
print("Sentiment for the product is - ", product_class.loc[item_name][0])
type(product_class.loc[item_name])

Item name Craft Punch Giga Scallop Circle 45 24687534 To 334
Craft Punch Giga Scallop Circle 45 24687534 To 334
Sentiment for the product is -  1.0


pandas.core.series.Series

In [162]:
groupedDf.get_group('Craft Punch Giga Scallop Circle 45 24687534 To 334')

,name,reviews_rating,reviews_username,user_sentiment,reviews_tokenized,sentiment_class
1262,Craft Punch Giga Scallop Circle 45 24687534 To...,5,j9,Positive,very easy on the hands to use i used it on car...,1


In [163]:
top20UserBased

name
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd                                  6.269380
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)                              5.679494
Clorox Disinfecting Bathroom Cleaner                                                             3.651637
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                                             3.370093
The Resident Evil Collection 5 Discs (blu-Ray)                                                   3.370093
Tostitos Bite Size Tortilla Chips                                                                3.370093
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                                        3.370093
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo                                         3.370093
Chester's Cheese Flavored Puffcorn Snacks                                                        3.333333
Tostitos Simply Blue Corn Tortilla Chips 

### Overwrite the top20 item ratings with the average sentiment ranging from 1 to 5.

In [164]:
for itmName in list(top20UserBased.index):
    top20UserBased[itmName] = product_class.loc[itmName][0]

### Top 5 items picked by sentiments.

In [165]:
top20UserBased.sort_values(ascending=False)[:5]

name
Tostitos Simply Blue Corn Tortilla Chips                                                         0.952381
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                                             0.914671
Hoover174 Platinum Collection153 Lightweight Bagged Upright Vacuum With Canister - Uh30010com    0.895028
Clorox Disinfecting Bathroom Cleaner                                                             0.880824
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)                              0.880140
Name: abby, dtype: float64

##### END OF FILE